Run the below four cells to obtain all libraries and mount drive if your dataset is stored in google drive.

#Import Models and libraries

In [1]:
!nvidia-smi

Fri Sep 25 07:32:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd drive/My\ Drive/Text\ Graphs\ 2020\ Final\ Folder/

/content/drive/My Drive/Text Graphs 2020 Final Folder


In [4]:
!pip install transformers
import nltk
nltk.download('stopwords')
from pathlib import Path
from tqdm.notebook import tqdm
from tqdm import trange
import pandas as po
import numpy as np
import warnings
import pickle
import nltk
import math
from paths import get_path_q, get_path_df_scores, get_path_predict
from utils import get_df_explanations, get_questions, obtain_model_names_and_classes, obtain_useful
from rank import get_ranks, get_preds, ideal_rerank, remove_combo_suffix, format_predict_line
from train_and_eval_functions import train_model, evaluate_model, predict_model
SEP = "#" * 100 + "\n"
path_data   = Path("data/")
path_tables = path_data.joinpath("raw/tables")
device = 'cuda'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Train Model

Run the following set of cells to train the model. Specify the model name and model type from the list of options provided to you. Make sure the specific set of models are available of training.


####This will create new intermidiate variables to be used


In [ ]:
df, df_exp, uids, uid2idx, uid2text, ranks, preds  = obtain_useful(path_data, path_tables, mode='train',top_k = 100)

####This will save all the intermediate variables to be used

In [ ]:
with open('./data/df.pkl', 'wb') as f:
  pickle.dump(df, f)
with open('./data/df_exp.pkl', 'wb') as f:
  pickle.dump(df_exp, f)
with open('./data/uids.pkl', 'wb') as f:
  pickle.dump(uids, f)
with open('./data/uid2idx.pkl', 'wb') as f:
  pickle.dump(uid2idx, f)
with open('./data/uid2text.pkl', 'wb') as f:
  pickle.dump(uid2text, f)
with open('./data/ranks.pkl', 'wb') as f:
  pickle.dump(ranks, f)
with open('./data/preds.pkl', 'wb') as f:
  pickle.dump(preds, f)

####This will load all the intermediate variables to be used

In [ ]:
with open('./data/df.pkl', 'rb') as f:
  df= pickle.load( f)
with open('./data/df_exp.pkl', 'rb') as f:
  df_exp= pickle.load(f)
with open('./data/uids.pkl', 'rb') as f:
  uids = pickle.load(f)
with open('./data/uid2idx.pkl', 'rb') as f:
  uid2idx = pickle.load(f)
with open('./data/uid2text.pkl', 'rb') as f:
  uid2text = pickle.load(f)
with open('./data/ranks.pkl', 'rb') as f:
  ranks = pickle.load(f)
with open('./data/preds.pkl', 'rb') as f:
  preds = pickle.load(f)

## Run this to train the model

In [ ]:
MODEL_CLASSES, model_with_no_token_types, tokenizer = obtain_model_names_and_classes(model_name='roberta', model_type='roberta-base')

Choose from the list of models and their respective pretrained versions:
[{'bert': ['bert-base-uncased']}, {'xlm': ['xlm-mlm-en-2048']}, {'roberta': ['roberta-base']}, {'bart': ['facebook/bart-base']}, {'longformer': ['allenai/longformer-base-4096']}, {'albert': ['albert-xxlarge-v2']}, {'electra': ['google/electra-large-generator']}, {'reformer': ['google/reformer-crime-and-punishment']}, {'mobilebert': ['mobile']}, {'scibert': ['allenai/scibert_scivocab_uncased']}]


In [ ]:
save_model= train_model( df, df_exp, uids, uid2idx, uid2text, ranks, preds, MODEL_CLASSES,
                        model_with_no_token_types, model_name='roberta', model_type='roberta-base', 
                        top_k = 100, num_train_epochs = 1, BATCH_SIZE=128, learning_rate=2e-5, epsilon=1e-8, 
                        gradient_accumulation_steps = 1, max_grad_norm = 1, weight_decay = 0, number_of_warmup_steps=0, 
                        global_step = 0, tr_loss=0.0, logging_loss = 0.0 )

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out



Model is saved as :  roberta_t_100_epoc_1_lr2e-05b_s128
Use this to load the model


#Train MAP
You can run this cell after training the model or directly by loading the appropriate saved models and then evaluating the model to get the MAP score on Train DATASET.

####This will create new intermidiate variables to be used


In [9]:
df, df_exp, uids, uid2idx, uid2text, ranks, preds  = obtain_useful(path_data, path_tables, mode='train',top_k = 100)

 15%|█▌        | 1368/9029 [00:00<00:00, 13670.85it/s]

Default lemmatizer (41883) concatenated (or not) with extras (162) -> 41883


100%|██████████| 9029/9029 [00:00<00:00, 14583.83it/s]


Explanation df shape: (9029, 5)


 41%|████▏     | 915/2207 [00:00<00:00, 9142.16it/s]

...

100%|██████████| 2207/2207 [00:00<00:00, 9085.66it/s]


(2207, 23)
Num canonical explanations: 9029
Num canonical explanations: 9029


100%|██████████| 2207/2207 [07:39<00:00,  4.80it/s]


Num canonical explanations: 9029


100%|██████████| 2207/2207 [00:00<00:00, 11715.10it/s]


####This will save all the intermediate variables to be used

In [ ]:
with open('./data/df.pkl', 'wb') as f:
  pickle.dump(df, f)
with open('./data/df_exp.pkl', 'wb') as f:
  pickle.dump(df_exp, f)
with open('./data/uids.pkl', 'wb') as f:
  pickle.dump(uids, f)
with open('./data/uid2idx.pkl', 'wb') as f:
  pickle.dump(uid2idx, f)
with open('./data/uid2text.pkl', 'wb') as f:
  pickle.dump(uid2text, f)
with open('./data/ranks.pkl', 'wb') as f:
  pickle.dump(ranks, f)
with open('./data/preds.pkl', 'wb') as f:
  pickle.dump(preds, f)

####This will load all the intermediate variables to be used

In [5]:
with open('./data/df.pkl', 'rb') as f:
  df= pickle.load( f)
with open('./data/df_exp.pkl', 'rb') as f:
  df_exp= pickle.load(f)
with open('./data/uids.pkl', 'rb') as f:
  uids = pickle.load(f)
with open('./data/uid2idx.pkl', 'rb') as f:
  uid2idx = pickle.load(f)
with open('./data/uid2text.pkl', 'rb') as f:
  uid2text = pickle.load(f)
with open('./data/ranks.pkl', 'rb') as f:
  ranks = pickle.load(f)
with open('./data/preds.pkl', 'rb') as f:
  preds = pickle.load(f)

## Run this to get MAP on train

In [7]:
## Evaluate Model on train dataset
MODEL_CLASSES, model_with_no_token_types, tokenizer = obtain_model_names_and_classes(model_name='roberta', model_type='roberta-base')
save_model='bart_t_100_epoc_1_lr_2e-05_b_s_256'
evaluate_model(tokenizer,df, df_exp, uids, uid2idx, uid2text, ranks, preds, save_model, model_with_no_token_types, model_name='roberta', mode='train', top_k = 100 )

Choose from the list of models and their respective pretrained versions:
[{'bert': ['bert-base-uncased']}, {'xlm': ['xlm-mlm-en-2048']}, {'roberta': ['roberta-base']}, {'bart': ['facebook/bart-base']}, {'longformer': ['allenai/longformer-base-4096']}, {'albert': ['albert-xxlarge-v2']}, {'electra': ['google/electra-large-generator']}, {'reformer': ['google/reformer-crime-and-punishment']}, {'mobilebert': ['mobile']}, {'scibert': ['allenai/scibert_scivocab_uncased']}]


FileNotFoundError: ignored

#Dev MAP
Running this cell below will give you the dev MAP score of the given Model  

####This will create new intermidiate variables to be used


In [5]:
df, df_exp, uids, uid2idx, uid2text, ranks, preds  = obtain_useful(path_data, path_tables, mode='dev',top_k = 100)

 16%|█▌        | 1421/9029 [00:00<00:00, 14203.88it/s]

Default lemmatizer (41883) concatenated (or not) with extras (162) -> 41883


100%|██████████| 496/496 [00:00<00:00, 9004.00it/s]


Explanation df shape: (9029, 5)
Default lemmatizer (41883) concatenated (or not) with extras (162) -> 41883
(496, 23)
Num canonical explanations: 9029
Num canonical explanations: 9029


100%|██████████| 496/496 [01:44<00:00,  4.75it/s]


Num canonical explanations: 9029


100%|██████████| 496/496 [00:00<00:00, 25518.87it/s]


####This will save all the intermediate variables to be used

In [ ]:
with open('./data/df.pkl', 'wb') as f:
  pickle.dump(df, f)
with open('./data/df_exp.pkl', 'wb') as f:
  pickle.dump(df_exp, f)
with open('./data/uids.pkl', 'wb') as f:
  pickle.dump(uids, f)
with open('./data/uid2idx.pkl', 'wb') as f:
  pickle.dump(uid2idx, f)
with open('./data/uid2text.pkl', 'wb') as f:
  pickle.dump(uid2text, f)
with open('./data/ranks.pkl', 'wb') as f:
  pickle.dump(ranks, f)
with open('./data/preds.pkl', 'wb') as f:
  pickle.dump(preds, f)

####This will load all the intermediate variables to be used

In [ ]:
with open('./data/df.pkl', 'rb') as f:
  df= pickle.load( f)
with open('./data/df_exp.pkl', 'rb') as f:
  df_exp= pickle.load(f)
with open('./data/uids.pkl', 'rb') as f:
  uids = pickle.load(f)
with open('./data/uid2idx.pkl', 'rb') as f:
  uid2idx = pickle.load(f)
with open('./data/uid2text.pkl', 'rb') as f:
  uid2text = pickle.load(f)
with open('./data/ranks.pkl', 'rb') as f:
  ranks = pickle.load(f)
with open('./data/preds.pkl', 'rb') as f:
  preds = pickle.load(f)

## Run this to get MAP on dev

In [8]:
## Evaluate Model on train dataset
MODEL_CLASSES, model_with_no_token_types, tokenizer = obtain_model_names_and_classes(model_name='roberta', model_type='roberta-base')
save_model='roberta_t_100_epoc_1_lr_2e-05_b_s_128'
evaluate_model(tokenizer,df, df_exp, uids, uid2idx, uid2text, ranks, preds, save_model, model_with_no_token_types, model_name='roberta', mode='dev', top_k = 100 )

Choose from the list of models and their respective pretrained versions:
[{'bert': ['bert-base-uncased']}, {'xlm': ['xlm-mlm-en-2048']}, {'roberta': ['roberta-base']}, {'bart': ['facebook/bart-base']}, {'longformer': ['allenai/longformer-base-4096']}, {'albert': ['albert-xxlarge-v2']}, {'electra': ['google/electra-large-generator']}, {'reformer': ['google/reformer-crime-and-punishment']}, {'mobilebert': ['mobile']}, {'scibert': ['allenai/scibert_scivocab_uncased']}]



0.4742807289780093
0.4562901561051407


#Test predictions
Running this cell below will give 

In [ ]:
## Get Test predictions 
df, ranks, preds = obtain_for_eval(mode='test',top_k = 500, path_tables,path_data)
predict_model(tokenizer,df, df_exp, uids, uid2idx, uid2text, ranks, preds, save_model, model_with_no_token_types, top_k=100, model_name='roberta')